# Feature Engineering

This notebook will engineer text features from the property reviews, and temporal features from calendar data.  The text features will include the top BOW words from the preprocessed text, and cluster labels after text embedding.  (May need extra computing power).  The temporal features will be aggregated features like average price of neighbourhood by month.

In [1]:
# Read in libraries
# Load libraries
import numpy as np
import pandas as pd
import time
import os
import sys
import ast
import json
import random
import datetime as dt
from datetime import datetime
from itertools import chain
import yellowbrick
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.cluster import DBSCAN, KMeans
import string
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
#from deep_translator import GoogleTranslator
import langid
import holidays
import datetime

# Unlimited columns
pd.options.display.max_columns = None

In [2]:
# Read in data
# Load calendar data
calendar = pd.read_csv("../data/processed/calendar.csv")

# Load reviews data
#review_map = pd.read_csv("../data/raw/reviews_ids.csv")
reviews = pd.read_csv("../data/processed/reviews.csv")

# Load listings data
listings = pd.read_csv("../data/processed/listings.csv")

# Load neighborhoods
neighborhoods = pd.read_csv("../data/processed/neighbourhoods.csv")

In [ ]:
# Functions
# Description:  Turn into BOW, and keep top n words OHE
def preprocess_text(text):
    """
    
    """
    
    stop_words = list(set(stopwords.words('english')))
    punctuation = string.punctuation
    stop_words += list(punctuation)
    stop_words.extend(['``','’', '`','br','"',"”", "''", "'s", "/b"]) 
    text = text.replace(r'<br />',' ')
    preprocessed = []    
        
    # Tokenization using nltk word tokenization
    tokenized = word_tokenize(text)
    for token in tokenized:
        token = token.lower()
        if token not in stop_words and len(token) > 1:
            preprocessed.append(token)
    return " ".join(preprocessed)

In [ ]:
def plot_pca_clusters(
    data,
    cluster_labels,
    raw_sents,
    show_labels=False,
    size=100,
    title="PCA visualization",
):
    """
    Carry out dimensionality reduction using PCA and plot 2-dimensional clusters.

    Parameters
    -----------
    data : numpy array
        data as a numpy array
    cluster_labels : list
        cluster labels for each row in the dataset
    raw_sents : list
        the original raw sentences for labeling datapoints
    show_labels : boolean
        whether you want to show labels for points or not (default: False)
    size : int
        size of points in the scatterplot
    title : str
        title for the visualization plot

    Returns
    -----------
    None. Shows the clusters.
    """

    pca = PCA(n_components=2)
    principal_comp = pca.fit_transform(data)
    pca_df = pd.DataFrame(data=principal_comp, columns=["pca1", "pca2"])
    pca_df["cluster"] = cluster_labels
    
    plt.figure(figsize=(10, 7))
    plt.title(title)
    ax = sns.scatterplot(
        x="pca1", y="pca2", hue="cluster", data=pca_df, palette="tab10", s=size
    )

    x = pca_df["pca1"].tolist()
    y = pca_df["pca2"].tolist()
    if show_labels:
        for i, txt in enumerate(raw_sents):
            plt.annotate(" ".join(txt.split()[:10]), (x[i], y[i]))
        ax.legend(loc="upper left")

    plt.show()

## Review Feature Engineering

In [ ]:
# Create smaller dataset for testing code
#random.seed(42)
#test_ids = random.sample(list(set(reviews.listing_id)), 100)
#test_data = reviews.query("listing_id in @test_ids")

#### Remove empty string, remove extra characters

In [ ]:
# Turn nan values into empty strings
reviews.comments.replace(np.nan, "", inplace=True)

# preprocess text
reviews.assign(comments = reviews.comments.apply(preprocess_text))

## Translate to English

The time taken to translate all non-english comments is roughly 12.2 hours, so non-english comments will be removed for now.  Translation to happen at a later point.

In [ ]:
# Find the language of each comment
#test_data['language_tuple'] = test_data.comments.apply(langid.classify)
#test_data[['lang', 'cert']] = pd.DataFrame(test_data['language_tuple'].tolist(), index=test_data.index)
#test_data.drop(['language_tuple', 'cert'], axis=1, inplace=True)

# Get non-English comments
#non_english_comments = test_data.query("lang != 'en'").comments.tolist()

# tranlate the comments
#non_english_comments = GoogleTranslator(source='auto', target='en').translate_batch(non_english_comments, )

# Replace the non-english comments with tranlated versioins
#test_data.loc[test_data.lang != 'en', 'comments'] = non_english_comments

#### Remove non-english comments

In [ ]:
# Find the language of each comment
reviews['language_tuple'] = reviews.comments.apply(langid.classify)
reviews[['lang', 'cert']] = pd.DataFrame(reviews['language_tuple'].tolist(), index=reviews.index)
reviews.drop(['language_tuple', 'cert'], axis=1, inplace=True)
reviews = reviews.query("lang == 'en'")

#### Add binary bow features for most common words in reviews

In [ ]:
# BOW
# Create count vectorizer object.  Max features can be changed
desc_n = 15
vec = CountVectorizer(binary=True, max_features=desc_n)

# Fit count vectorizer
X_counts = vec.fit_transform(reviews.comments)

# Create new features
bow_df = pd.DataFrame(X_counts.toarray(), columns=["rev_" + str(col) for col in list(vec.vocabulary_.keys())], index=reviews.index)

# Add them onto the original dataframe
reviews = reviews.join(bow_df)

#### Use NLTK's sentiment intensity analyzer to determine review sentiment.  Save score for pos, neg, neu.

In [ ]:
# Sentiment Analysis
# https://www.nltk.org/howto/sentiment.html
sia = SentimentIntensityAnalyzer()

pos_scores = []
neg_scores = []
neu_scores = []
for rev in reviews.comments:
    pos_scores.append(sia.polarity_scores(rev)['pos'])
    neg_scores.append(sia.polarity_scores(rev)['neg'])
    neu_scores.append(sia.polarity_scores(rev)['neu'])
    
reviews = reviews.assign(rev_pos_score = pos_scores, 
                 rev_neg_score = neg_scores,
                 rev_neu_score = neu_scores)

#### Cluster reviews, one hot encode cluster assignments

In [ ]:
embedder = SentenceTransformer("paraphrase-distilroberta-base-v1")

In [ ]:
comment_data = reviews['comments'].tolist()
embeddings = embedder.encode(comment_data)
comment_embeddings = pd.DataFrame(
    embeddings,
    index=reviews.index,
)

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(3, 10))

visualizer.fit(comment_embeddings)  # Fit the data to the visualizer
visualizer.show();

In [ ]:
np.random.seed(42)
km_labels_dict = {
    k: KMeans(k).fit(comment_embeddings).predict(comment_embeddings) for k in np.arange(2, 8)
}

In [ ]:
for k, labels in km_labels_dict.items():
    plot_pca_clusters(
        comment_embeddings,
        labels,
        reviews['comments'],
        size=10,
        title="KMeans with sentence embeddings (k=%d)" % (k),
    )

In [ ]:
# Clustering
k = 4
reviews['review_cluster'] = km_labels_dict[k]

In [ ]:
reviews.groupby('review_cluster').sample(5)

In [ ]:
reviews.groupby(['review_cluster'])[['rev_pos_score', 'rev_neg_score', 'rev_neu_score']].mean()

**Summary:** Overall, the clusters don't seem helpful.

In [ ]:
# Add new features
reviews.head()

In [ ]:
# Comments
reviews['comment_length'] = reviews['comments'].str.split(" ").apply(len)

In [ ]:
reviews.columns

In [ ]:
# Define aggregations
aggregation_dict = {'reviewer_id': 'count',
                    'rev_clean': 'mean',
                    'rev_place': 'mean',
                    'rev_stay' : 'mean',
                    'rev_would': 'mean',
                    'rev_nice' : 'mean',
                    'rev_great' : 'mean',
                    'rev_easy' : 'mean',
                    'rev_host' : 'mean',
                    'rev_location' : 'mean',
                    'rev_everything' : 'mean',
                    'rev_comfortable' : 'mean',
                    'rev_recommend' : 'mean',
                    'rev_recommend' : 'mean',
                    'rev_room' : 'mean',
                    'rev_pos_score' : 'mean',
                    'rev_neg_score' : 'mean',
                    'rev_neu_score' : 'mean',
                    'review_cluster' : pd.Series.mode,
                    'comment_length' : 'mean',
                    'rev_neu_score' : 'mean',
                    'rev_neu_score' : 'mean'}

In [ ]:
# Average positivity, neutral, and negative scores
agg_reviews = reviews.groupby("listing_id")
agg_reviews = agg_reviews.agg(aggregation_dict)

In [ ]:
agg_names = {"reviewer_id" : "review_count",
             "rev_clean" : "word_clean_use",
             "rev_place" : "word_place_use",
             "rev_stay" : "word_stay_use",
             "rev_would" : "word_would_use",
             "rev_nice" : "word_nice_use",
             "rev_great" : "word_great_use",
             "rev_easy" : "word_easy_use",
             "rev_host" : "word_host_use",
             "rev_location" : "word_location_use",
             "rev_everything" : "word_everything_use",
             "rev_comfortable" : "word_comfortable_use",
             "rev_recommend" : "word_recommend_use",
             "rev_room" : "word_room_use",
             "rev_pos_score" : "positive_score_mean",
             "rev_neg_score" : "negative_score_mean",
             "rev_neu_score" : "neutral_score_mean",
             "review_cluster" : "most_common_review_cluster",
             "comment_length" : "average_comment_length"}

agg_reviews.rename(columns=agg_names, inplace=True)

In [ ]:
agg_reviews = agg_reviews.reset_index()

In [ ]:
# Merge relevant listing data
agg_reviews = agg_reviews.merge(listings[['id', 'review_span']], left_on='listing_id', right_on='id').drop(columns=['id'])

# Add reviews per review span
agg_reviews = agg_reviews.assign(review_frequency = agg_reviews.review_count / agg_reviews.review_span)

In [ ]:
agg_reviews = agg_reviews.drop(columns='review_span')

In [ ]:
agg_reviews.head()

## Calendar Feature Engineering

In [ ]:
# Merge neighbourhood data onto calendar data
calendar.head()

In [ ]:
# Coerce date to datetime
calendar['date'] = pd.to_datetime(calendar['date'], errors='coerce')

In [ ]:
# Add day of week feature
calendar = calendar.assign(day_of_week = calendar['date'].dt.dayofweek)

In [ ]:
# Add week of year feature
calendar = calendar.assign(week_of_year = calendar['date'].dt.isocalendar().week)

In [ ]:
# Add month of year feature
calendar = calendar.assign(month_of_year = calendar['date'].dt.month)

In [ ]:
# Add year feature
calendar = calendar.assign(year = calendar['date'].dt.year)

In [ ]:
# Add binary holiday feature
# Get US holidays
us_ca_holidays = holidays.country_holidays('US', subdiv='CA')

# Format holidays
us_ca_holidays = pd.DataFrame({"holiday":
    [#datetime.date(2018, 1, 1),
     #datetime.date(2018, 1, 15),
     #datetime.date(2018, 2, 14),
     #datetime.date(2018, 2, 19),
     #datetime.date(2018, 3, 17),
     #datetime.date(2018, 5, 5),
     #datetime.date(2018, 5, 28),
     #datetime.date(2018, 7, 4),
     #datetime.date(2018, 8, 10),
     #datetime.date(2018, 8, 11),
     #datetime.date(2018, 8, 12),
     #datetime.date(2018, 9, 3),
     #datetime.date(2018, 10, 8),
     #datetime.date(2018, 10, 31),
     #datetime.date(2018, 11, 11),
     #datetime.date(2018, 11, 12),
     #datetime.date(2018, 11, 22),
     #datetime.date(2018, 12, 25),
     #datetime.date(2019, 1, 1),
     #datetime.date(2019, 1, 15),
     #datetime.date(2019, 2, 14),
     #datetime.date(2019, 2, 19),
     #datetime.date(2019, 3, 17),
     #datetime.date(2019, 5, 5),
     #datetime.date(2019, 5, 28),
     #datetime.date(2019, 7, 4),
     #datetime.date(2019, 8, 9),
     #datetime.date(2019, 8, 10),
     #datetime.date(2019, 8, 11),
     #datetime.date(2019, 9, 3),
     #datetime.date(2019, 10, 8),
     #datetime.date(2019, 10, 31),
     #datetime.date(2019, 11, 11),
     #datetime.date(2019, 11, 12),
     #datetime.date(2019, 11, 22),
     #datetime.date(2019, 12, 25),
     ##datetime.date(2020, 1, 1),
     #datetime.date(2020, 1, 15),
     #datetime.date(2020, 2, 14),
     #datetime.date(2020, 2, 19),
     #datetime.date(2020, 3, 17),
     #datetime.date(2020, 5, 5),
     #datetime.date(2020, 5, 28),
     #datetime.date(2020, 7, 4),
     #datetime.date(2020, 9, 3),
     #datetime.date(2020, 10, 8),
     datetime.date(2020, 10, 31),
     datetime.date(2020, 11, 11),
     datetime.date(2020, 11, 12),
     datetime.date(2020, 11, 22),
     datetime.date(2020, 12, 25),
     datetime.date(2021, 1, 1),
     datetime.date(2021, 1, 15),
     datetime.date(2021, 2, 14),
     datetime.date(2021, 2, 19),
     datetime.date(2021, 3, 17),
     datetime.date(2021, 5, 5),
     datetime.date(2021, 5, 28),
     datetime.date(2021, 7, 4),
     datetime.date(2021, 9, 3),
     datetime.date(2021, 10, 8),
     datetime.date(2021, 10, 29),
     datetime.date(2021, 10, 30),
     datetime.date(2021, 10, 31),
     datetime.date(2021, 11, 11),
     datetime.date(2021, 11, 12),
     datetime.date(2021, 11, 22),
     datetime.date(2021, 12, 25),
     datetime.date(2022, 1, 1),
     datetime.date(2022, 1, 15),
     datetime.date(2022, 2, 14),
     datetime.date(2022, 2, 19),
     datetime.date(2022, 3, 17),
     datetime.date(2022, 5, 5),
     datetime.date(2022, 5, 28),
     datetime.date(2022, 7, 4),
     datetime.date(2022, 8, 5),
     datetime.date(2022, 8, 6),
     datetime.date(2022, 8, 7),
     datetime.date(2022, 9, 3),
     datetime.date(2022, 10, 8),
     datetime.date(2022, 10, 31),
     datetime.date(2022, 11, 11),
     datetime.date(2022, 11, 12),
     datetime.date(2022, 11, 22),
     datetime.date(2022, 12, 25)]}
)

us_ca_holidays['holiday'] = pd.to_datetime(us_ca_holidays['holiday'], errors='coerce')

In [ ]:
#def nearest_holiday(date_series, holidays):
#    min_holiday_dist = []
#    for i, date in enumerate(date_series):
#        min_holiday_dist.append(min(abs(holidays - date)).days)
#    return(min_holiday_dist)

In [ ]:
holidays = [date.date() for date in us_ca_holidays['holiday']]

In [ ]:
calendar = calendar.assign(holiday = [True if date.date() in holidays else False for date in calendar.date])

In [ ]:
calendar.head()

In [ ]:
calendar_duration_agg = calendar.groupby('listing_id')[['available', 'minimum_nights', 'maximum_nights']].mean()

In [ ]:
# day of week aggregations
dow_agg = calendar.groupby(['listing_id', 'day_of_week'])[['price']].mean()
dow_agg = dow_agg.reset_index()
dow_pivot = dow_agg.pivot(index='listing_id', columns='day_of_week', values='price').reset_index()
dow_columns = ['listing_id'] + ['day_of_week_' + str(i) for i in dow_pivot.columns[1:]]
dow_pivot.columns = dow_columns

# week of year aggregations
woy_agg = calendar.groupby(['listing_id', 'week_of_year'])[['price']].mean()
woy_agg = woy_agg.reset_index()
woy_pivot = woy_agg.pivot(index='listing_id', columns='week_of_year', values='price').reset_index()
woy_columns = ['listing_id'] + ['week_of_year_' + str(i) for i in woy_pivot.columns[1:]]
woy_pivot.columns = woy_columns

# day of week aggregations
moy_agg = calendar.groupby(['listing_id', 'month_of_year'])[['price']].mean()
moy_agg = moy_agg.reset_index()
moy_pivot = moy_agg.pivot(index='listing_id', columns='month_of_year', values='price').reset_index()
moy_columns = ['listing_id'] + ['month_of_year_' + str(i) for i in moy_pivot.columns[1:]]
moy_pivot.columns = moy_columns

# day of week aggregations
year_agg = calendar.groupby(['listing_id', 'year'])[['price']].mean()
year_agg = year_agg.reset_index()
year_pivot = year_agg.pivot(index='listing_id', columns='year', values='price').reset_index()
year_columns = ['listing_id'] + ['year_' + str(i) for i in year_pivot.columns[1:]]
year_pivot.columns = year_columns

# holiday aggregations
holiday_agg = calendar.groupby(['listing_id', 'holiday'])[['price']].mean()
holiday_agg = holiday_agg.reset_index()
holiday_pivot = holiday_agg.pivot(index='listing_id', columns='holiday', values='price').reset_index()
holiday_columns = ['listing_id'] + ['holiday_' + str(i) for i in holiday_pivot.columns[1:]]
holiday_pivot.columns = holiday_columns

# Merge together
calendar_aggregates = dow_pivot.merge(woy_pivot, on='listing_id').merge(moy_pivot, on='listing_id').merge(year_pivot, on='listing_id').merge(holiday_pivot, on='listing_id')

In [ ]:
calendar_aggregates = calendar_aggregates.merge(calendar_duration_agg, on='listing_id')

In [ ]:
calendar_aggregates.head()

## Listings Feature Engineering

In [ ]:
listings.head()

In [ ]:
modeling_columns = ['id',
                     'host_id',
                     'host_response_time',
                     'host_response_rate',
                     'host_acceptance_rate',
                     'host_is_superhost',
                     'host_neighbourhood',
                     'host_listings_count',
                     'host_total_listings_count',
                     'host_has_profile_pic',
                     'host_identity_verified',
                     'neighbourhood_cleansed',
                     'neighbourhood_group_cleansed',
                     'latitude',
                     'longitude',
                     'property_type',
                     'room_type',
                     'accommodates',
                     'bedrooms',
                     'beds',
                     'price',
                     'minimum_nights',
                     'maximum_nights',
                     'minimum_minimum_nights',
                     'maximum_minimum_nights',
                     'minimum_maximum_nights',
                     'maximum_maximum_nights',
                     'minimum_nights_avg_ntm',
                     'maximum_nights_avg_ntm',
                     'has_availability',
                     'availability_30',
                     'availability_60',
                     'availability_90',
                     'availability_365',
                     'number_of_reviews',
                     'number_of_reviews_ltm',
                     'number_of_reviews_l30d',
                     'review_scores_rating',
                     'review_scores_accuracy',
                     'review_scores_cleanliness',
                     'review_scores_checkin',
                     'review_scores_communication',
                     'review_scores_location',
                     'review_scores_value',
                     'instant_bookable',
                     'calculated_host_listings_count',
                     'calculated_host_listings_count_entire_homes',
                     'calculated_host_listings_count_private_rooms',
                     'calculated_host_listings_count_shared_rooms',
                     'reviews_per_month',
                     'desc_apartment',
                     'desc_located',
                     'desc_space',
                     'desc_home',
                     'desc_bed',
                     'desc_room',
                     'desc_kitchen',
                     'desc_access',
                     'desc_one',
                     'desc_private',
                     'desc_san',
                     'desc_francisco',
                     'desc_bathroom',
                     'desc_bedroom',
                     'desc_living',
                     'host_in_sf',
                     'host_verifications_email',
                     'host_verifications_facebook',
                     'host_verifications_google',
                     'host_verifications_government_id',
                     'host_verifications_identity_manual',
                     'host_verifications_jumio',
                     'host_verifications_kba',
                     'host_verifications_manual_offline',
                     'host_verifications_manual_online',
                     'host_verifications_offline_government_id',
                     'host_verifications_phone',
                     'host_verifications_reviews',
                     'host_verifications_selfie',
                     'host_verifications_sent_id',
                     'host_verifications_work_email',
                     'host_verifications_zhima_selfie',
                     'bathroom_private',
                     'bathroom_shared',
                     'bathroom_half',
                     'bathroom_count',
                     'amenities_Wifi',
                     'amenities_Smoke alarm',
                     'amenities_Essentials',
                     'amenities_Heating',
                     'amenities_Hangers',
                     'amenities_Carbon monoxide alarm',
                     'amenities_Hair dryer',
                     'amenities_Iron',
                     'amenities_Long term stays allowed',
                     'amenities_Kitchen',
                     'amenities_Shampoo',
                     'amenities_Dedicated workspace',
                     'amenities_Hot water',
                     'amenities_Washer',
                     'amenities_Fire extinguisher',
                     'amenities_Dryer',
                     'amenities_Coffee maker',
                     'amenities_Refrigerator',
                     'amenities_Microwave',
                     'amenities_Dishes and silverware',
                     'amenities_Bed linens',
                     'amenities_TV',
                     'amenities_Cooking basics',
                     'amenities_First aid kit',
                     'amenities_Private entrance',
                     'amenities_Free street parking',
                     'amenities_Oven',
                     'amenities_Stove',
                     'amenities_Extra pillows and blankets',
                     'amenities_Dishwasher',
                     'review_span',
                     't_since_last_review',
                     't_as_host',
                     'has_license']

In [ ]:
# Get the desired features
listings_modeling = listings[modeling_columns]

In [ ]:
# Rename columns
listings_modeling.rename(columns={'id' : 'listing_id', 'price' : 'listing_price'}, inplace=True)

In [ ]:
listings_modeling.head()

## Create the Modeling Data

In [ ]:
calendar.head()

In [ ]:
base = calendar[['listing_id', 'date', 'price']]

In [ ]:
base = base.merge(agg_reviews, on='listing_id').merge(calendar_aggregates, on='listing_id').merge(listings_modeling, on='listing_id')

In [ ]:
base.head()

In [ ]:
base.shape

## Split into Train-Test

In [ ]:
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

In [ ]:
np.random.seed(42)

# Create smaller dataset for testing code
train_fraction = 0.8
number_of_listings = len(set(base.listing_id))

train_size = round(train_fraction * number_of_listings)
train_ids = random.sample(list(set(base.listing_id)), train_size)
test_ids = [listing for listing in base.listing_id if listing not in train_ids]

train_data = base.query("listing_id in @train_ids")
test_data = base.query("listing_id in @test_ids")

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
len(set(train_data.listing_id))

In [ ]:
len(set(test_data.listing_id))

In [ ]:
# Set output path
out_path = "../data/ready_for_modeling/"

In [ ]:
# Make processed folder
#if ~os.direxists("../data/ready_for_modeling"):
#    os.mkdir("../data/ready_for_modeling")
try:
    os.mkdir("../data/ready_for_modeling")
except:
    print("Processed directory exists")

In [ ]:
# Write to disc
train_path = out_path + "training_data.csv"
train_data.to_csv(train_path)

test_path = out_path + "testing_data.csv"
test_data.to_csv(test_path)